In [ ]:
!pip install tensorboardX
!pip install torchnet
!pip install datasets

# Diffusion Model Trainer

This notebook performs classification model.

In [2]:
import os
colab = True

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir('/content/drive/MyDrive/Tesis/tesis/3_classifier')
else:
    os.chdir('../3_classifier')

Mounted at /content/drive


In [3]:
import time

from tqdm import *

import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler

import torchvision
from torchvision.transforms import *

from tensorboardX import SummaryWriter

import models
from datasets.speech_commands_dataset import *
from transforms import transforms_wav as twav
from transforms import transforms_stft as tstft
from mixup import *

In [4]:
''' Load parameters like we did in the training script '''
from collections import namedtuple

Args = namedtuple('Args', [
    'train_dataset',
    'valid_dataset',
    'background_noise',
    'comment',
    'batch_size',
    'dataload_workers_nums',
    'weight_decay',
    'optim',
    'learning_rate',
    'lr_scheduler',
    'lr_scheduler_patience',
    'lr_scheduler_step_size',
    'lr_scheduler_gamma',
    'max_epochs',
    # 'resume',
    'model',
    'input',
    'mixup'
  ])

args = Args(
    train_dataset='../data/classifier/train.csv',
    valid_dataset='../data/classifier/val.csv',
    background_noise='datasets/_background_noise_',
    comment='',
    batch_size=96,
    dataload_workers_nums=6,
    weight_decay=1e-3,
    optim='sgd',
    learning_rate=0.01,
    lr_scheduler='plateau',
    lr_scheduler_patience=5,
    lr_scheduler_step_size=50,
    lr_scheduler_gamma=0.1,
    max_epochs=70,
    # resume='',
    model='resnext29_8_64',
    input='mel32',
    mixup=False
    )

In [5]:
use_gpu = torch.cuda.is_available()
print('use_gpu', use_gpu)
if use_gpu:
    torch.backends.cudnn.benchmark = True
n_mels = 32

use_gpu True


In [6]:
data_aug_transform = Compose([
    # twav.ChangeAmplitude(),
    # twav.ChangeSpeedAndPitchAudio(),
    twav.FixAudioLength(),
    tstft.ToSTFT(),
    # tstft.StretchAudioOnSTFT(),
    # tstft.TimeshiftAudioOnSTFT(),
    tstft.FixSTFTDimension()])
bg_dataset = BackgroundNoiseDataset(args.background_noise, data_aug_transform)
add_bg_noise = tstft.AddBackgroundNoiseOnSTFT(bg_dataset)
train_feature_transform = Compose([
    tstft.ToMelSpectrogramFromSTFT(n_mels=n_mels),
    tstft.DeleteSTFT(),
    twav.ToTensor('mel_spectrogram', 'input')])
train_dataset = SpeechCommandsDataset(args.train_dataset,
                                Compose([twav.LoadAudio(),
                                         data_aug_transform,
                                         add_bg_noise,
                                         train_feature_transform]))

valid_feature_transform = Compose([
    twav.ToMelSpectrogram(n_mels=n_mels),
    twav.ToTensor('mel_spectrogram', 'input')])
valid_dataset = SpeechCommandsDataset(args.valid_dataset,
                                Compose([twav.LoadAudio(),
                                         twav.FixAudioLength(),
                                         valid_feature_transform]))

weights = train_dataset.make_weights_for_balanced_classes()
sampler = WeightedRandomSampler(weights, len(weights))
train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, sampler=sampler, drop_last=True)
                            #   pin_memory=use_gpu, num_workers=args.dataload_workers_nums)
valid_dataloader = DataLoader(valid_dataset, batch_size=args.batch_size, shuffle=False)
                            #   pin_memory=use_gpu, num_workers=args.dataload_workers_nums)


# a name used to save checkpoints etc.
full_name = '%s_%s_%s_bs%d_lr%.1e_wd%.1e' % (args.model, args.optim, args.lr_scheduler, args.batch_size, args.learning_rate, args.weight_decay)
if args.comment:
    full_name = '%s_%s' % (full_name, args.comment)

model = models.create_model(model_name=args.model, num_classes=len(train_dataset.classes), in_channels=1)

if use_gpu:
    model = torch.nn.DataParallel(model).cuda()

criterion = torch.nn.CrossEntropyLoss()

if args.optim == 'sgd':
    optimizer = torch.optim.SGD(model.parameters(), lr=args.learning_rate, momentum=0.9, weight_decay=args.weight_decay)
else:
    optimizer = torch.optim.Adam(model.parameters(), lr=args.learning_rate)

start_timestamp = int(time.time()*1000)
start_epoch = 0
best_accuracy = 0
best_loss = 1e100
global_step = 0


if args.lr_scheduler == 'plateau':
    lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=args.lr_scheduler_patience, factor=args.lr_scheduler_gamma)
else:
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=args.lr_scheduler_step_size, gamma=args.lr_scheduler_gamma, last_epoch=start_epoch-1)

def get_lr():
    return optimizer.param_groups[0]['lr']

writer = SummaryWriter(comment=('_speech_commands_' + full_name))



/content/drive/MyDrive/Tesis/tesis/classificador/models/resnext.py:99: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.classifier.weight)
/content/drive/MyDrive/Tesis/tesis/classificador/models/resnext.py:104: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.state_dict()[key], mode='fan_out')


In [7]:
next(model.parameters()).device

device(type='cuda', index=0)

In [8]:
def train(epoch):
    global global_step

    print("epoch %3d with lr=%.02e" % (epoch, get_lr()))
    phase = 'train'
    writer.add_scalar('%s/learning_rate' % phase,  get_lr(), epoch)

    model.train()  # Set model to training mode

    running_loss = 0.0
    it = 0
    correct = 0
    total = 0

    pbar = tqdm(train_dataloader, unit="audios", unit_scale=train_dataloader.batch_size)
    for batch in pbar:
        inputs = batch['input']
        inputs = torch.unsqueeze(inputs, 1)
        targets = batch['target']

        if args.mixup:
            inputs, targets = mixup(inputs, targets, num_classes=len(train_dataset.classes))

        inputs = Variable(inputs, requires_grad=True)
        targets = Variable(targets, requires_grad=False)

        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        # forward/backward
        outputs = model(inputs)
        if args.mixup:
            loss = mixup_cross_entropy_loss(outputs, targets)
        else:
            loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # statistics
        it += 1
        global_step += 1
        # print("--------------------------------------------------------------------", loss.data)
        running_loss += loss.data.item()
        pred = outputs.data.max(1, keepdim=True)[1]
        if args.mixup:
            targets = batch['target']
            targets = Variable(targets, requires_grad=False).cuda()
        correct += pred.eq(targets.data.view_as(pred)).sum()
        total += targets.size(0)

        writer.add_scalar('%s/loss' % phase, loss.data.item(), global_step)

        # update the progress bar
        pbar.set_postfix({
            'loss': "%.05f" % (running_loss / it),
            'acc': "%.02f%%" % (100*correct/total)
        })

    accuracy = correct/total
    epoch_loss = running_loss / it
    writer.add_scalar('%s/accuracy' % phase, 100*accuracy, epoch)
    writer.add_scalar('%s/epoch_loss' % phase, epoch_loss, epoch)

    return epoch_loss

def valid(epoch):
    global best_accuracy, best_loss, global_step

    phase = 'valid'
    model.eval()  # Set model to evaluate mode

    running_loss = 0.0
    it = 0
    correct = 0
    total = 0

    pbar = tqdm(valid_dataloader, unit="audios", unit_scale=valid_dataloader.batch_size)
    for batch in pbar:
        inputs = batch['input']
        inputs = torch.unsqueeze(inputs, 1)
        targets = batch['target']

        with torch.no_grad():
            inputs = Variable(inputs, volatile = True)
        targets = Variable(targets, requires_grad=False)

        if use_gpu:
            inputs = inputs.cuda()
            targets = targets.cuda()

        # forward
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # statistics
        it += 1
        global_step += 1
        running_loss += loss.data.item()
        pred = outputs.data.max(1, keepdim=True)[1]
        correct += pred.eq(targets.data.view_as(pred)).sum()
        total += targets.size(0)

        writer.add_scalar('%s/loss' % phase, loss.data.item(), global_step)

        # update the progress bar
        pbar.set_postfix({
            'loss': "%.05f" % (running_loss / it),
            'acc': "%.02f%%" % (100*correct/total)
        })

    accuracy = correct/total
    epoch_loss = running_loss / it
    writer.add_scalar('%s/accuracy' % phase, 100*accuracy, epoch)
    writer.add_scalar('%s/epoch_loss' % phase, epoch_loss, epoch)

    checkpoint = {
        'epoch': epoch,
        'step': global_step,
        'state_dict': model.state_dict(),
        'loss': epoch_loss,
        'accuracy': accuracy,
        'optimizer' : optimizer.state_dict(),
    }

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(checkpoint, 'checkpoints/best-loss-speech-commands-checkpoint-%s.pth' % full_name)
        torch.save(model, '%d-%s-best-loss-no_dataaug.pth' % (start_timestamp, full_name))
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        torch.save(checkpoint, 'checkpoints/best-acc-speech-commands-checkpoint-%s.pth' % full_name)
        torch.save(model, '%d-%s-best-acc-no-dataaug.pth' % (start_timestamp, full_name))

    torch.save(checkpoint, 'checkpoints/last-speech-commands-checkpoint.pth')
    del checkpoint  # reduce memory

    return epoch_loss


In [ ]:
print("training %s for Google speech commands..." % args.model)
since = time.time()
for epoch in range(start_epoch, args.max_epochs):
    if args.lr_scheduler == 'step':
        lr_scheduler.step()

    train(epoch)
    epoch_loss = valid(epoch)

    if args.lr_scheduler == 'plateau':
        lr_scheduler.step(metrics=epoch_loss)

    time_elapsed = time.time() - since
    time_str = 'total time elapsed: {:.0f}h {:.0f}m {:.0f}s '.format(time_elapsed // 3600, time_elapsed % 3600 // 60, time_elapsed % 60)
    print("%s, best accuracy: %.02f%%, best loss %f" % (time_str, 100*best_accuracy, best_loss))
print("finished")

training resnext29_8_64 for Google speech commands...
epoch   0 with lr=1.00e-02


  0%|          | 0/4704 [00:00<?, ?audios/s]<ipython-input-8-80f31247c5de>:86: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(inputs, volatile = True)
100%|██████████| 4704/4704 [50:57<00:00,  1.54audios/s, loss=0.02526, acc=99.31%]


total time elapsed: 3h 18m 31s , best accuracy: 99.31%, best loss 0.025262
epoch   1 with lr=1.00e-02


100%|██████████| 4704/4704 [00:41<00:00, 113.70audios/s, loss=0.01624, acc=99.40%]


total time elapsed: 4h 13m 16s , best accuracy: 99.40%, best loss 0.016240
epoch   2 with lr=1.00e-02


...

100%|██████████| 4704/4704 [00:41<00:00, 112.01audios/s, loss=0.00453, acc=99.91%]


total time elapsed: 7h 21m 24s , best accuracy: 99.94%, best loss 0.003144
epoch  35 with lr=1.00e-05


100%|██████████| 4704/4704 [00:41<00:00, 112.06audios/s, loss=0.00461, acc=99.91%]


total time elapsed: 7h 26m 12s , best accuracy: 99.94%, best loss 0.003144
epoch  36 with lr=1.00e-05


 85%|████████▍ | 15744/18528 [03:29<00:37, 74.31audios/s, loss=0.05787, acc=97.05%]